In [1]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

### Congo

In [269]:
import pandas as pd
df = pd.read_csv('short_prompt_responses_Congolese_employee_1.csv',error_bad_lines=False)
df1 = pd.read_csv('short_prompt_responses_Congolese_employee_2.csv',error_bad_lines=False)
df2 = pd.read_csv('short_prompt_responses_Congolese_international student_1.csv',error_bad_lines=False)
df3 = pd.read_csv('short_prompt_responses_Congolese_international student_2.csv',error_bad_lines=False)

<ipython-input-269-65b4530face3>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('short_prompt_responses_Congolese_employee_1.csv',error_bad_lines=False)
<ipython-input-269-65b4530face3>:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df1 = pd.read_csv('short_prompt_responses_Congolese_employee_2.csv',error_bad_lines=False)
<ipython-input-269-65b4530face3>:4: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df2 = pd.read_csv('short_prompt_responses_Congolese_international student_1.csv',error_bad_lines=False)
<ipython-input-269-65b4530face3>:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df3 = p

In [270]:
import re
import string

def remove_links(tweet):
    '''Takes a string and removes web links from it'''
    tweet = re.sub(r'http\S+', '', tweet) # remove http links
    tweet = re.sub(r'bit.ly/\S+', '', tweet) # rempve bitly links
    tweet = tweet.strip('[link]') # remove [links]
    return tweet

def remove_users(tweet):
    '''Takes a string and removes retweet and @user information'''
    tweet = re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet) # remove retweet
    tweet = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet) # remove tweeted at
    return tweet

# Add custom words to stopwords list
additional_stopwords = {'also', '...', 'etc', 'often', ',','keep','might','morning','always','yet','find','make','sense','wake'}
my_stopwords = set(stopwords.words('english')) | additional_stopwords
# word_rooter = nltk.stem.snowball.PorterStemmer(ignore_stopwords=False).stem
my_punctuation = '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~•@'


# Define a translation table to remove punctuation
translator = str.maketrans('', '', string.punctuation)

def clean_tweet(tweet, bigrams=False):
    tweet = remove_users(tweet)
    tweet = remove_links(tweet)
    tweet = tweet.lower()  # Convert to lower case
    tweet = tweet.translate(translator)  # Remove punctuation
    tweet = re.sub('\s+', ' ', tweet)  # Remove double spacing
    tweet = re.sub('([0-9]+)', '', tweet)  # Remove numbers
    tweet_token_list = [word for word in tweet.split(' ')
                        if word not in my_stopwords]  # Remove stopwords
    # stemming algorithm sometimes cut off the end of the words which may lead to inaccuracy.
    # tweet_token_list = [word_rooter(word) if '#' not in word else word
    #                     for word in tweet_token_list] # apply word rooter

    if bigrams:
        tweet_token_list = tweet_token_list+[tweet_token_list[i]+'_'+tweet_token_list[i+1]
                                            for i in range(len(tweet_token_list)-1)]

    tweet = ' '.join(tweet_token_list)
    return tweet

# Apply the cleaning function to your dataframe
df['clean_tweet'] = df['response'].apply(clean_tweet)
df1['clean_tweet'] = df1['response'].apply(clean_tweet)
df2['clean_tweet'] = df2['response'].apply(clean_tweet)
df3['clean_tweet'] = df3['response'].apply(clean_tweet)

In [271]:
from sklearn.feature_extraction.text import CountVectorizer

# Adjusted token pattern
token_pattern = r'\b[a-zA-Z]{3,}\b'  # Only words with 3 or more letters

vectorizer = CountVectorizer(max_df=0.9, min_df=0.1, token_pattern=token_pattern)


# apply transformation
tf = vectorizer.fit_transform(df['clean_tweet']).toarray() #xth row te yth word thakle 1, 0-1 matrix

# tf_feature_names tells us what word each column in the matric represents
tf_feature_names = vectorizer.get_feature_names_out() #yth word as a list


In [272]:
len(tf[0].tolist())


479

In [273]:
tf_feature_names[20]

'arts'

In [274]:
from sklearn.decomposition import LatentDirichletAllocation

number_of_topics = 5

model = LatentDirichletAllocation(n_components=number_of_topics, random_state=0)


In [275]:
model.fit(tf)


LatentDirichletAllocation(n_components=5, random_state=0)

In [276]:
def display_topics(model, feature_names, no_top_words):
    topic_dict = {}
    for topic_idx, topic in enumerate(model.components_):
        topic_dict["Topic %d words" % (topic_idx)]= ['{}'.format(feature_names[i])
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
        topic_dict["Topic %d weights" % (topic_idx)]= ['{:.1f}'.format(topic[i])
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
    return pd.DataFrame(topic_dict)

no_top_words = 20
print("short_prompt_responses_Congolese_employee_1:")
display_topics(model, tf_feature_names, no_top_words)


short_prompt_responses_Congolese_employee_1:


,Topic 0 words,Topic 0 weights,Topic 1 words,Topic 1 weights,Topic 2 words,Topic 2 weights,Topic 3 words,Topic 3 weights,Topic 4 words,Topic 4 weights
0,country,3.2,work,12.2,challenge,0.2,work,3.2,challenge,0.2
1,see,3.2,community,4.2,growth,0.2,culture,2.2,growth,0.2
2,learned,2.2,colleagues,4.2,vibrant,0.2,rich,2.2,vibrant,0.2
3,cultural,2.2,face,3.2,workplace,0.2,beloved,2.2,workplace,0.2
4,resilience,2.2,deep,3.2,development,0.2,challenge,1.2,development,0.2
5,perspective,2.2,people,3.2,home,0.2,growth,1.2,home,0.2
6,remain,2.2,homeland,3.2,using,0.2,vibrant,1.2,using,0.2
7,future,2.2,hard,3.2,proud,0.2,whether,1.2,proud,0.2
8,proud,1.2,strength,3.2,carry,0.2,runs,1.2,carry,0.2
9,using,1.2,value,3.2,humble,0.2,family,1.2,humble,0.2


In [277]:
from sklearn.feature_extraction.text import CountVectorizer

# Adjusted token pattern
token_pattern = r'\b[a-zA-Z]{3,}\b'  # Only words with 3 or more letters

vectorizer = CountVectorizer(max_df=0.9, min_df=0.1, token_pattern=token_pattern)


# apply transformation
tf1 = vectorizer.fit_transform(df1['clean_tweet']).toarray() #xth row te yth word thakle 1, 0-1 matrix

# tf_feature_names tells us what word each column in the matric represents
tf_feature_names = vectorizer.get_feature_names_out() #yth word as a list

In [278]:
len(tf1[0].tolist())

186

In [279]:
tf_feature_names[20]

'capacity'

In [280]:
from sklearn.decomposition import LatentDirichletAllocation

number_of_topics = 5

model1 = LatentDirichletAllocation(n_components=number_of_topics, random_state=0)

model1.fit(tf1)

no_top_words = 20
print("short_prompt_responses_Congolese_employee_2:")
display_topics(model1, tf_feature_names, no_top_words)

short_prompt_responses_Congolese_employee_2:


,Topic 0 words,Topic 0 weights,Topic 1 words,Topic 1 weights,Topic 2 words,Topic 2 weights,Topic 3 words,Topic 3 weights,Topic 4 words,Topic 4 weights
0,resilience,0.2,individual,2.2,unique,2.2,may,4.2,may,7.2
1,individual,0.2,education,2.2,working,2.2,one,2.2,like,3.2
2,healthcare,0.2,either,2.2,cultural,2.2,greatly,1.2,depending,3.2
3,agriculture,0.2,culture,2.2,likely,1.2,challenges,1.2,diverse,2.2
4,adaptability,0.2,drc,2.2,specific,1.2,adaptability,1.2,job,2.2
5,home,0.2,home,1.2,speak,1.2,home,1.2,possess,2.2
6,country,0.2,adaptability,1.2,technology,1.2,perspectives,1.2,employees,2.2
7,skills,0.2,agriculture,1.2,agriculture,1.2,respect,1.2,experience,2.2
8,education,0.2,healthcare,1.2,healthcare,1.2,background,1.2,ethic,2.2
9,could,0.2,local,1.2,background,1.2,bring,1.2,business,2.2


In [281]:
from sklearn.feature_extraction.text import CountVectorizer

# Adjusted token pattern
token_pattern = r'\b[a-zA-Z]{3,}\b'  # Only words with 3 or more letters

vectorizer = CountVectorizer(max_df=0.9, min_df=0.1, token_pattern=token_pattern)


# apply transformation
tf2 = vectorizer.fit_transform(df2['clean_tweet']).toarray() #xth row te yth word thakle 1, 0-1 matrix

# tf_feature_names tells us what word each column in the matric represents
tf_feature_names = vectorizer.get_feature_names_out() #yth word as a list

In [282]:
len(tf2[0].tolist())

609

In [283]:
tf_feature_names[20]

'ahead'

In [284]:
from sklearn.decomposition import LatentDirichletAllocation

number_of_topics = 5

model2 = LatentDirichletAllocation(n_components=number_of_topics, random_state=0)

model2.fit(tf2)

no_top_words = 20
print("short_prompt_responses_Congolese_international student_1:")
display_topics(model2, tf_feature_names, no_top_words)

short_prompt_responses_Congolese_international student_1:


,Topic 0 words,Topic 0 weights,Topic 1 words,Topic 1 weights,Topic 2 words,Topic 2 weights,Topic 3 words,Topic 3 weights,Topic 4 words,Topic 4 weights
0,different,3.2,culture,4.2,democratic,0.2,culture,3.2,back,3.2
1,back,3.2,heart,3.2,cultures,0.2,every,3.2,experiences,2.2
2,theres,3.2,africa,3.2,republic,0.2,country,2.2,country,2.2
3,family,2.2,like,3.2,experiences,0.2,foreign,2.2,lingala,2.2
4,miles,2.2,stories,3.2,world,0.2,understanding,2.2,kinshasa,2.2
5,music,2.2,every,3.2,country,0.2,away,2.2,blend,2.2
6,thousands,2.2,vibrant,2.2,vibrant,0.2,knowledge,2.2,abroad,2.2
7,cultural,2.2,world,2.2,different,0.2,different,2.2,lands,2.2
8,dance,2.2,roots,2.2,back,0.2,carry,2.2,canada,2.2
9,identity,2.2,learn,2.2,one,0.2,heritage,2.2,congos,2.2


In [285]:
from sklearn.feature_extraction.text import CountVectorizer

# Adjusted token pattern
token_pattern = r'\b[a-zA-Z]{3,}\b'  # Only words with 3 or more letters

vectorizer = CountVectorizer(max_df=0.9, min_df=0.1, token_pattern=token_pattern)


# apply transformation
tf3 = vectorizer.fit_transform(df3['clean_tweet']).toarray() #xth row te yth word thakle 1, 0-1 matrix

# tf_feature_names tells us what word each column in the matric represents
tf_feature_names = vectorizer.get_feature_names_out() #yth word as a list

In [286]:
len(tf3[0].tolist())

254

In [287]:
tf_feature_names[20]

'arts'

In [288]:
from sklearn.decomposition import LatentDirichletAllocation

number_of_topics = 5

model3 = LatentDirichletAllocation(n_components=number_of_topics, random_state=0)

model3.fit(tf3)

no_top_words = 20
print("short_prompt_responses_Congolese_international student_2:")
display_topics(model3, tf_feature_names, no_top_words)

short_prompt_responses_Congolese_international student_2:


,Topic 0 words,Topic 0 weights,Topic 1 words,Topic 1 weights,Topic 2 words,Topic 2 weights,Topic 3 words,Topic 3 weights,Topic 4 words,Topic 4 weights
0,individual,4.2,resilient,0.2,likely,4.2,resilient,0.2,known,3.2
1,african,4.2,community,0.2,country,3.2,community,0.2,rich,2.2
2,students,3.2,english,0.2,social,3.2,english,0.2,global,2.2
3,education,3.2,determined,0.2,determined,2.2,determined,0.2,music,2.2
4,culture,3.2,country,0.2,homesickness,2.2,country,0.2,cultural,2.2
5,unique,2.2,could,0.2,ambitious,2.2,could,0.2,ethnic,2.2
6,issues,2.2,adapting,0.2,kikongo,2.2,adapting,0.2,diversity,2.2
7,like,2.2,foreign,0.2,tshiluba,2.2,foreign,0.2,educational,2.2
8,speak,2.2,unique,0.2,background,2.2,unique,0.2,would,2.2
9,experiences,2.2,abroad,0.2,many,2.2,abroad,0.2,resilient,1.2


### China

In [289]:
import pandas as pd
df = pd.read_csv('short_prompt_responses_Chinese_employee_1.csv',error_bad_lines=False)
df1 = pd.read_csv('short_prompt_responses_Chinese_employee_2.csv',error_bad_lines=False)
df2 = pd.read_csv('short_prompt_responses_Chinese_international student_1.csv',error_bad_lines=False)
df3 = pd.read_csv('short_prompt_responses_Chinese_international student_2.csv',error_bad_lines=False)

<ipython-input-289-691bb6baf83b>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('short_prompt_responses_Chinese_employee_1.csv',error_bad_lines=False)
<ipython-input-289-691bb6baf83b>:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df1 = pd.read_csv('short_prompt_responses_Chinese_employee_2.csv',error_bad_lines=False)
<ipython-input-289-691bb6baf83b>:4: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df2 = pd.read_csv('short_prompt_responses_Chinese_international student_1.csv',error_bad_lines=False)
<ipython-input-289-691bb6baf83b>:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df3 = pd.read

In [290]:
import re
import string

def remove_links(tweet):
    '''Takes a string and removes web links from it'''
    tweet = re.sub(r'http\S+', '', tweet) # remove http links
    tweet = re.sub(r'bit.ly/\S+', '', tweet) # rempve bitly links
    tweet = tweet.strip('[link]') # remove [links]
    return tweet

def remove_users(tweet):
    '''Takes a string and removes retweet and @user information'''
    tweet = re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet) # remove retweet
    tweet = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet) # remove tweeted at
    return tweet

# Add custom words to stopwords list
additional_stopwords = {'also', '...', 'etc', 'often', ',','keep','might','morning','always','yet','find','make','sense','wake'}
my_stopwords = set(stopwords.words('english')) | additional_stopwords
# word_rooter = nltk.stem.snowball.PorterStemmer(ignore_stopwords=False).stem
my_punctuation = '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~•@'


# Define a translation table to remove punctuation
translator = str.maketrans('', '', string.punctuation)

def clean_tweet(tweet, bigrams=False):
    tweet = remove_users(tweet)
    tweet = remove_links(tweet)
    tweet = tweet.lower()  # Convert to lower case
    tweet = tweet.translate(translator)  # Remove punctuation
    tweet = re.sub('\s+', ' ', tweet)  # Remove double spacing
    tweet = re.sub('([0-9]+)', '', tweet)  # Remove numbers
    tweet_token_list = [word for word in tweet.split(' ')
                        if word not in my_stopwords]  # Remove stopwords
    # stemming algorithm sometimes cut off the end of the words which may lead to inaccuracy.
    # tweet_token_list = [word_rooter(word) if '#' not in word else word
    #                     for word in tweet_token_list] # apply word rooter

    if bigrams:
        tweet_token_list = tweet_token_list+[tweet_token_list[i]+'_'+tweet_token_list[i+1]
                                            for i in range(len(tweet_token_list)-1)]

    tweet = ' '.join(tweet_token_list)
    return tweet

# Apply the cleaning function to your dataframe
df['clean_tweet'] = df['response'].apply(clean_tweet)
df1['clean_tweet'] = df1['response'].apply(clean_tweet)
df2['clean_tweet'] = df2['response'].apply(clean_tweet)
df3['clean_tweet'] = df3['response'].apply(clean_tweet)

In [291]:
from sklearn.feature_extraction.text import CountVectorizer

# Adjusted token pattern
token_pattern = r'\b[a-zA-Z]{3,}\b'  # Only words with 3 or more letters

vectorizer = CountVectorizer(max_df=0.9, min_df=0.1, token_pattern=token_pattern)


# apply transformation
tf = vectorizer.fit_transform(df['clean_tweet']).toarray() #xth row te yth word thakle 1, 0-1 matrix

# tf_feature_names tells us what word each column in the matric represents
tf_feature_names = vectorizer.get_feature_names_out() #yth word as a list


In [292]:
len(tf[0].tolist())


440

In [293]:
tf_feature_names[20]

'another'

In [294]:
from sklearn.decomposition import LatentDirichletAllocation

number_of_topics = 5

model = LatentDirichletAllocation(n_components=number_of_topics, random_state=0)


In [295]:
model.fit(tf)


LatentDirichletAllocation(n_components=5, random_state=0)

In [296]:
def display_topics(model, feature_names, no_top_words):
    topic_dict = {}
    for topic_idx, topic in enumerate(model.components_):
        topic_dict["Topic %d words" % (topic_idx)]= ['{}'.format(feature_names[i])
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
        topic_dict["Topic %d weights" % (topic_idx)]= ['{:.1f}'.format(topic[i])
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
    return pd.DataFrame(topic_dict)

no_top_words = 20
print("short_prompt_responses_Chinese_employee_1:")
display_topics(model, tf_feature_names, no_top_words)


short_prompt_responses_Chinese_employee_1:


,Topic 0 words,Topic 0 weights,Topic 1 words,Topic 1 weights,Topic 2 words,Topic 2 weights,Topic 3 words,Topic 3 weights,Topic 4 words,Topic 4 weights
0,relationships,2.2,professional,3.2,culture,3.2,early,0.2,highly,2.2
1,diligence,2.2,working,3.2,superiors,2.2,every,0.2,early,1.2
2,life,2.2,culture,3.2,hierarchy,2.2,professional,0.2,values,1.2
3,hand,2.2,every,2.2,cultural,2.2,values,0.2,brainstorming,1.2
4,family,2.2,value,2.2,experience,2.2,relationships,0.2,linguistic,1.2
5,culture,2.2,skills,2.2,nature,2.2,days,0.2,positively,1.2
6,pride,1.2,dedication,2.2,knowledge,2.2,hours,0.2,hard,1.2
7,hours,1.2,days,2.2,worker,2.2,pride,0.2,meticulous,1.2
8,one,1.2,suit,2.2,pay,2.2,colleagues,0.2,uphold,1.2
9,sun,1.2,essential,2.2,life,2.2,value,0.2,contribute,1.2


In [297]:
from sklearn.feature_extraction.text import CountVectorizer

# Adjusted token pattern
token_pattern = r'\b[a-zA-Z]{3,}\b'  # Only words with 3 or more letters

vectorizer = CountVectorizer(max_df=0.9, min_df=0.1, token_pattern=token_pattern)


# apply transformation
tf1 = vectorizer.fit_transform(df1['clean_tweet']).toarray() #xth row te yth word thakle 1, 0-1 matrix

# tf_feature_names tells us what word each column in the matric represents
tf_feature_names = vectorizer.get_feature_names_out() #yth word as a list

In [298]:
len(tf1[0].tolist())

200

In [299]:
tf_feature_names[20]

'careful'

In [300]:
from sklearn.decomposition import LatentDirichletAllocation

number_of_topics = 5

model1 = LatentDirichletAllocation(n_components=number_of_topics, random_state=0)

model1.fit(tf1)

no_top_words = 20
print("short_prompt_responses_Chinese_employee_2:")
display_topics(model1, tf_feature_names, no_top_words)

short_prompt_responses_Chinese_employee_2:


,Topic 0 words,Topic 0 weights,Topic 1 words,Topic 1 weights,Topic 2 words,Topic 2 weights,Topic 3 words,Topic 3 weights,Topic 4 words,Topic 4 weights
0,business,3.2,hardworking,0.2,hardworking,0.2,could,4.2,may,2.2
1,english,2.2,may,0.2,may,0.2,employees,3.2,hardworking,1.2
2,mandarin,2.2,chinas,0.2,chinas,0.2,may,3.2,chinas,1.2
3,cantonese,2.2,workforce,0.2,workforce,0.2,traits,3.2,workforce,1.2
4,great,2.2,great,0.2,great,0.2,diligent,2.2,consider,1.2
5,ethic,2.2,nationality,0.2,nationality,0.2,strong,2.2,deeply,1.2
6,respect,2.2,detailoriented,0.2,detailoriented,0.2,many,2.2,workplace,1.2
7,china,2.2,respect,0.2,respect,0.2,nationality,2.2,part,1.2
8,likely,2.2,strong,0.2,strong,0.2,skills,2.2,known,1.2
9,tend,2.2,however,0.2,however,0.2,possess,2.2,styles,1.2


In [301]:
from sklearn.feature_extraction.text import CountVectorizer

# Adjusted token pattern
token_pattern = r'\b[a-zA-Z]{3,}\b'  # Only words with 3 or more letters

vectorizer = CountVectorizer(max_df=0.9, min_df=0.1, token_pattern=token_pattern)


# apply transformation
tf2 = vectorizer.fit_transform(df2['clean_tweet']).toarray() #xth row te yth word thakle 1, 0-1 matrix

# tf_feature_names tells us what word each column in the matric represents
tf_feature_names = vectorizer.get_feature_names_out() #yth word as a list

In [302]:
len(tf2[0].tolist())

614

In [303]:
tf_feature_names[20]

'alone'

In [304]:
from sklearn.decomposition import LatentDirichletAllocation

number_of_topics = 5

model2 = LatentDirichletAllocation(n_components=number_of_topics, random_state=0)

model2.fit(tf2)

no_top_words = 20
print("short_prompt_responses_Chinese_international student_1:")
display_topics(model2, tf_feature_names, no_top_words)

short_prompt_responses_Chinese_international student_1:


,Topic 0 words,Topic 0 weights,Topic 1 words,Topic 1 weights,Topic 2 words,Topic 2 weights,Topic 3 words,Topic 3 weights,Topic 4 words,Topic 4 weights
0,grown,0.2,grown,0.2,making,4.2,cultural,5.2,social,4.2
1,help,0.2,help,0.2,china,3.2,language,5.2,university,3.2
2,ive,0.2,ive,0.2,food,2.2,china,5.2,ive,2.2
3,challenge,0.2,challenge,0.2,ive,2.2,different,4.2,beijing,2.2
4,opportunity,0.2,opportunity,0.2,students,2.2,studying,4.2,cultures,2.2
5,back,0.2,back,0.2,western,2.2,back,4.2,part,2.2
6,world,0.2,world,0.2,sharing,2.2,enjoy,3.2,time,2.2
7,especially,0.2,especially,0.2,cultures,2.2,exploring,3.2,life,2.2
8,environment,0.2,environment,0.2,journey,2.2,balance,3.2,isnt,2.2
9,currently,0.2,currently,0.2,however,2.2,foreign,3.2,appreciate,2.2


In [305]:
from sklearn.feature_extraction.text import CountVectorizer

# Adjusted token pattern
token_pattern = r'\b[a-zA-Z]{3,}\b'  # Only words with 3 or more letters

vectorizer = CountVectorizer(max_df=0.9, min_df=0.1, token_pattern=token_pattern)


# apply transformation
tf3 = vectorizer.fit_transform(df3['clean_tweet']).toarray() #xth row te yth word thakle 1, 0-1 matrix

# tf_feature_names tells us what word each column in the matric represents
tf_feature_names = vectorizer.get_feature_names_out() #yth word as a list

In [306]:
len(tf3[0].tolist())

345

In [307]:
tf_feature_names[20]

'applicable'

In [308]:
from sklearn.decomposition import LatentDirichletAllocation

number_of_topics = 5

model3 = LatentDirichletAllocation(n_components=number_of_topics, random_state=0)

model3.fit(tf3)

no_top_words = 20
print("short_prompt_responses_Chinese_international student_2:")
display_topics(model3, tf_feature_names, no_top_words)

short_prompt_responses_Chinese_international student_2:


,Topic 0 words,Topic 0 weights,Topic 1 words,Topic 1 weights,Topic 2 words,Topic 2 weights,Topic 3 words,Topic 3 weights,Topic 4 words,Topic 4 weights
0,many,2.2,students,6.2,may,7.2,environment,0.2,environment,0.2
1,new,2.2,may,6.2,academic,4.2,usually,0.2,usually,0.2
2,experience,2.2,education,4.2,home,4.2,barriers,0.2,barriers,0.2
3,students,2.2,different,4.2,family,4.2,many,0.2,many,0.2
4,barriers,1.2,host,4.2,students,4.2,pursue,0.2,pursue,0.2
5,pursue,1.2,mandarin,3.2,usually,3.2,hardworking,0.2,hardworking,0.2
6,learning,1.2,share,3.2,environment,3.2,ambassadors,0.2,ambassadors,0.2
7,experiencing,1.2,traditions,3.2,traditions,3.2,new,0.2,new,0.2
8,shock,1.2,adjusting,2.2,friends,3.2,academic,0.2,academic,0.2
9,seen,1.2,like,2.2,back,2.2,back,0.2,back,0.2


### France

In [309]:
import pandas as pd
df = pd.read_csv('short_prompt_responses_French_employee_1.csv',error_bad_lines=False)
df1 = pd.read_csv('short_prompt_responses_French_employee_2.csv',error_bad_lines=False)
df2 = pd.read_csv('short_prompt_responses_French_international student_1.csv',error_bad_lines=False)
df3 = pd.read_csv('short_prompt_responses_French_international student_2.csv',error_bad_lines=False)

<ipython-input-309-f786bc8aca8a>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('short_prompt_responses_French_employee_1.csv',error_bad_lines=False)
<ipython-input-309-f786bc8aca8a>:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df1 = pd.read_csv('short_prompt_responses_French_employee_2.csv',error_bad_lines=False)
<ipython-input-309-f786bc8aca8a>:4: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df2 = pd.read_csv('short_prompt_responses_French_international student_1.csv',error_bad_lines=False)
<ipython-input-309-f786bc8aca8a>:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df3 = pd.read_cs

In [310]:
import re
import string

def remove_links(tweet):
    '''Takes a string and removes web links from it'''
    tweet = re.sub(r'http\S+', '', tweet) # remove http links
    tweet = re.sub(r'bit.ly/\S+', '', tweet) # rempve bitly links
    tweet = tweet.strip('[link]') # remove [links]
    return tweet

def remove_users(tweet):
    '''Takes a string and removes retweet and @user information'''
    tweet = re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet) # remove retweet
    tweet = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet) # remove tweeted at
    return tweet

# Add custom words to stopwords list
additional_stopwords = {'also', '...', 'etc', 'often', ',','keep','might','morning','always','yet','find','make','sense','wake'}
my_stopwords = set(stopwords.words('english')) | additional_stopwords
# word_rooter = nltk.stem.snowball.PorterStemmer(ignore_stopwords=False).stem
my_punctuation = '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~•@'


# Define a translation table to remove punctuation
translator = str.maketrans('', '', string.punctuation)

def clean_tweet(tweet, bigrams=False):
    tweet = remove_users(tweet)
    tweet = remove_links(tweet)
    tweet = tweet.lower()  # Convert to lower case
    tweet = tweet.translate(translator)  # Remove punctuation
    tweet = re.sub('\s+', ' ', tweet)  # Remove double spacing
    tweet = re.sub('([0-9]+)', '', tweet)  # Remove numbers
    tweet_token_list = [word for word in tweet.split(' ')
                        if word not in my_stopwords]  # Remove stopwords
    # stemming algorithm sometimes cut off the end of the words which may lead to inaccuracy.
    # tweet_token_list = [word_rooter(word) if '#' not in word else word
    #                     for word in tweet_token_list] # apply word rooter

    if bigrams:
        tweet_token_list = tweet_token_list+[tweet_token_list[i]+'_'+tweet_token_list[i+1]
                                            for i in range(len(tweet_token_list)-1)]

    tweet = ' '.join(tweet_token_list)
    return tweet

# Apply the cleaning function to your dataframe
df['clean_tweet'] = df['response'].apply(clean_tweet)
df1['clean_tweet'] = df1['response'].apply(clean_tweet)
df2['clean_tweet'] = df2['response'].apply(clean_tweet)
df3['clean_tweet'] = df3['response'].apply(clean_tweet)

In [311]:
from sklearn.feature_extraction.text import CountVectorizer

# Adjusted token pattern
token_pattern = r'\b[a-zA-Z]{3,}\b'  # Only words with 3 or more letters

vectorizer = CountVectorizer(max_df=0.9, min_df=0.1, token_pattern=token_pattern)


# apply transformation
tf = vectorizer.fit_transform(df['clean_tweet']).toarray() #xth row te yth word thakle 1, 0-1 matrix

# tf_feature_names tells us what word each column in the matric represents
tf_feature_names = vectorizer.get_feature_names_out() #yth word as a list


In [312]:
len(tf[0].tolist())


488

In [313]:
tf_feature_names[20]

'appearance'

In [314]:
from sklearn.decomposition import LatentDirichletAllocation

number_of_topics = 5

model = LatentDirichletAllocation(n_components=number_of_topics, random_state=0)


In [315]:
model.fit(tf)


LatentDirichletAllocation(n_components=5, random_state=0)

In [316]:
def display_topics(model, feature_names, no_top_words):
    topic_dict = {}
    for topic_idx, topic in enumerate(model.components_):
        topic_dict["Topic %d words" % (topic_idx)]= ['{}'.format(feature_names[i])
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
        topic_dict["Topic %d weights" % (topic_idx)]= ['{:.1f}'.format(topic[i])
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
    return pd.DataFrame(topic_dict)

no_top_words = 20
print("short_prompt_responses_French_employee_1:")
display_topics(model, tf_feature_names, no_top_words)


short_prompt_responses_French_employee_1:


,Topic 0 words,Topic 0 weights,Topic 1 words,Topic 1 weights,Topic 2 words,Topic 2 weights,Topic 3 words,Topic 3 weights,Topic 4 words,Topic 4 weights
0,personal,5.2,believe,3.2,innovation,0.2,life,5.2,innovation,0.2
1,culture,4.2,culture,3.2,france,0.2,day,4.2,france,0.2
2,day,4.2,france,2.2,environment,0.2,even,3.2,environment,0.2
3,appreciate,3.2,pride,2.2,take,0.2,every,3.2,take,0.2
4,value,3.2,value,2.2,value,0.2,france,3.2,value,0.2
5,around,3.2,hours,2.2,known,0.2,usually,3.2,known,0.2
6,law,2.2,equality,2.2,uphold,0.2,theres,3.2,uphold,0.2
7,environment,2.2,workplace,2.2,heritage,0.2,personal,3.2,heritage,0.2
8,take,2.2,life,2.2,style,0.2,attention,2.2,style,0.2
9,activities,2.2,values,2.2,spirit,0.2,working,2.2,spirit,0.2


In [318]:
from sklearn.feature_extraction.text import CountVectorizer

# Adjusted token pattern
token_pattern = r'\b[a-zA-Z]{3,}\b'  # Only words with 3 or more letters

vectorizer = CountVectorizer(max_df=0.9, min_df=0.1, token_pattern=token_pattern)


# apply transformation
tf1 = vectorizer.fit_transform(df1['clean_tweet']).toarray() #xth row te yth word thakle 1, 0-1 matrix

# tf_feature_names tells us what word each column in the matric represents
tf_feature_names = vectorizer.get_feature_names_out() #yth word as a list

In [319]:
len(tf1[0].tolist())

219

In [320]:
tf_feature_names[20]

'business'

In [321]:
from sklearn.decomposition import LatentDirichletAllocation

number_of_topics = 5

model1 = LatentDirichletAllocation(n_components=number_of_topics, random_state=0)

model1.fit(tf1)

no_top_words = 20
print("short_prompt_responses_French_employee_2:")
display_topics(model1, tf_feature_names, no_top_words)

short_prompt_responses_French_employee_2:


,Topic 0 words,Topic 0 weights,Topic 1 words,Topic 1 weights,Topic 2 words,Topic 2 weights,Topic 3 words,Topic 3 weights,Topic 4 words,Topic 4 weights
0,work,3.2,strong,2.2,may,3.2,known,2.2,work,4.2
1,laws,2.2,english,1.2,could,2.2,country,2.2,good,3.2
2,labor,2.2,value,1.2,working,2.2,customs,1.2,typically,2.2
3,although,2.2,professional,1.2,dress,2.2,high,1.2,labor,2.2
4,ranging,1.2,vacation,1.2,leave,1.2,parental,1.2,generally,2.2
5,performance,1.2,time,1.2,like,1.2,languages,1.2,hours,2.2
6,personal,1.2,skills,1.2,tourism,1.2,possess,1.2,leaves,2.2
7,practice,1.2,ethics,1.2,depending,1.2,hour,1.2,paid,1.2
8,qualities,1.2,dedication,1.2,finance,1.2,leave,1.2,fluent,1.2
9,relocated,1.2,business,1.2,usually,1.2,paid,1.2,english,1.2


In [322]:
from sklearn.feature_extraction.text import CountVectorizer

# Adjusted token pattern
token_pattern = r'\b[a-zA-Z]{3,}\b'  # Only words with 3 or more letters

vectorizer = CountVectorizer(max_df=0.9, min_df=0.1, token_pattern=token_pattern)


# apply transformation
tf2 = vectorizer.fit_transform(df2['clean_tweet']).toarray() #xth row te yth word thakle 1, 0-1 matrix

# tf_feature_names tells us what word each column in the matric represents
tf_feature_names = vectorizer.get_feature_names_out() #yth word as a list

In [323]:
len(tf2[0].tolist())

658

In [324]:
tf_feature_names[20]

'allows'

In [325]:
from sklearn.decomposition import LatentDirichletAllocation

number_of_topics = 5

model2 = LatentDirichletAllocation(n_components=number_of_topics, random_state=0)

model2.fit(tf2)

no_top_words = 20
print("short_prompt_responses_French_international student_1:")
display_topics(model2, tf_feature_names, no_top_words)

short_prompt_responses_French_international student_1:


,Topic 0 words,Topic 0 weights,Topic 1 words,Topic 1 weights,Topic 2 words,Topic 2 weights,Topic 3 words,Topic 3 weights,Topic 4 words,Topic 4 weights
0,new,7.2,back,2.2,global,6.2,world,3.2,experiences,0.2
1,france,4.2,cultures,2.2,textbooks,3.2,share,2.2,charm,0.2
2,language,4.2,blend,2.2,experiences,2.2,people,2.2,learn,0.2
3,one,4.2,journey,2.2,charm,2.2,language,2.2,fields,0.2
4,every,4.2,home,2.2,academic,2.2,diverse,2.2,enriching,0.2
5,home,4.2,harmonious,2.2,world,2.2,romantic,2.2,experience,0.2
6,life,4.2,contemporary,2.2,one,2.2,literature,2.2,bridge,0.2
7,cultural,3.2,learn,1.2,education,2.2,coffee,2.2,baguettes,0.2
8,classic,3.2,fields,1.2,classroom,2.2,layers,2.2,languages,0.2
9,much,3.2,essence,1.2,life,2.2,global,2.2,culture,0.2


In [326]:
from sklearn.feature_extraction.text import CountVectorizer

# Adjusted token pattern
token_pattern = r'\b[a-zA-Z]{3,}\b'  # Only words with 3 or more letters

vectorizer = CountVectorizer(max_df=0.9, min_df=0.1, token_pattern=token_pattern)


# apply transformation
tf3 = vectorizer.fit_transform(df3['clean_tweet']).toarray() #xth row te yth word thakle 1, 0-1 matrix

# tf_feature_names tells us what word each column in the matric represents
tf_feature_names = vectorizer.get_feature_names_out() #yth word as a list

In [327]:
len(tf3[0].tolist())

265

In [328]:
tf_feature_names[20]

'arts'

In [329]:
from sklearn.decomposition import LatentDirichletAllocation

number_of_topics = 5

model3 = LatentDirichletAllocation(n_components=number_of_topics, random_state=0)

model3.fit(tf3)

no_top_words = 20
print("short_prompt_responses_French_international student_2:")
display_topics(model3, tf_feature_names, no_top_words)

short_prompt_responses_French_international student_2:


,Topic 0 words,Topic 0 weights,Topic 1 words,Topic 1 weights,Topic 2 words,Topic 2 weights,Topic 3 words,Topic 3 weights,Topic 4 words,Topic 4 weights
0,given,2.2,person,3.2,system,0.2,system,0.2,country,10.2
1,frances,2.2,cultural,2.2,strong,0.2,strong,0.2,culture,9.2
2,education,2.2,country,2.2,fluent,0.2,fluent,0.2,language,8.2
3,political,2.2,another,1.2,given,0.2,given,0.2,studying,7.2
4,show,2.2,someone,1.2,fashion,0.2,fashion,0.2,study,5.2
5,cultures,1.2,system,1.2,cultural,0.2,cultural,0.2,traditions,4.2
6,good,1.2,however,1.2,another,0.2,another,0.2,host,4.2
7,command,1.2,proficiency,1.2,someone,0.2,someone,0.2,cultural,4.2
8,arts,1.2,background,1.2,arts,0.2,arts,0.2,individual,3.2
9,system,1.2,knowledge,1.2,command,0.2,command,0.2,sharing,3.2


### Japan

In [330]:
import pandas as pd
df = pd.read_csv('short_prompt_responses_Japanese_employee_1.csv',error_bad_lines=False)
df1 = pd.read_csv('short_prompt_responses_Japanese_employee_2.csv',error_bad_lines=False)
df2 = pd.read_csv('short_prompt_responses_Japanese_international student_1.csv',error_bad_lines=False)
df3 = pd.read_csv('short_prompt_responses_Japanese_international student_2.csv',error_bad_lines=False)

<ipython-input-330-5c85090975f3>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('short_prompt_responses_Japanese_employee_1.csv',error_bad_lines=False)
<ipython-input-330-5c85090975f3>:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df1 = pd.read_csv('short_prompt_responses_Japanese_employee_2.csv',error_bad_lines=False)
<ipython-input-330-5c85090975f3>:4: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df2 = pd.read_csv('short_prompt_responses_Japanese_international student_1.csv',error_bad_lines=False)
<ipython-input-330-5c85090975f3>:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df3 = pd.r

In [331]:
import re
import string

def remove_links(tweet):
    '''Takes a string and removes web links from it'''
    tweet = re.sub(r'http\S+', '', tweet) # remove http links
    tweet = re.sub(r'bit.ly/\S+', '', tweet) # rempve bitly links
    tweet = tweet.strip('[link]') # remove [links]
    return tweet

def remove_users(tweet):
    '''Takes a string and removes retweet and @user information'''
    tweet = re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet) # remove retweet
    tweet = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet) # remove tweeted at
    return tweet

# Add custom words to stopwords list
additional_stopwords = {'also', '...', 'etc', 'often', ',','keep','might','morning','always','yet','find','make','sense','wake'}
my_stopwords = set(stopwords.words('english')) | additional_stopwords
# word_rooter = nltk.stem.snowball.PorterStemmer(ignore_stopwords=False).stem
my_punctuation = '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~•@'


# Define a translation table to remove punctuation
translator = str.maketrans('', '', string.punctuation)

def clean_tweet(tweet, bigrams=False):
    tweet = remove_users(tweet)
    tweet = remove_links(tweet)
    tweet = tweet.lower()  # Convert to lower case
    tweet = tweet.translate(translator)  # Remove punctuation
    tweet = re.sub('\s+', ' ', tweet)  # Remove double spacing
    tweet = re.sub('([0-9]+)', '', tweet)  # Remove numbers
    tweet_token_list = [word for word in tweet.split(' ')
                        if word not in my_stopwords]  # Remove stopwords
    # stemming algorithm sometimes cut off the end of the words which may lead to inaccuracy.
    # tweet_token_list = [word_rooter(word) if '#' not in word else word
    #                     for word in tweet_token_list] # apply word rooter

    if bigrams:
        tweet_token_list = tweet_token_list+[tweet_token_list[i]+'_'+tweet_token_list[i+1]
                                            for i in range(len(tweet_token_list)-1)]

    tweet = ' '.join(tweet_token_list)
    return tweet

# Apply the cleaning function to your dataframe
df['clean_tweet'] = df['response'].apply(clean_tweet)
df1['clean_tweet'] = df1['response'].apply(clean_tweet)
df2['clean_tweet'] = df2['response'].apply(clean_tweet)
df3['clean_tweet'] = df3['response'].apply(clean_tweet)

In [332]:
from sklearn.feature_extraction.text import CountVectorizer

# Adjusted token pattern
token_pattern = r'\b[a-zA-Z]{3,}\b'  # Only words with 3 or more letters

vectorizer = CountVectorizer(max_df=0.9, min_df=0.1, token_pattern=token_pattern)


# apply transformation
tf = vectorizer.fit_transform(df['clean_tweet']).toarray() #xth row te yth word thakle 1, 0-1 matrix

# tf_feature_names tells us what word each column in the matric represents
tf_feature_names = vectorizer.get_feature_names_out() #yth word as a list


In [333]:
len(tf[0].tolist())


521

In [334]:
tf_feature_names[20]

'appreciate'

In [335]:
from sklearn.decomposition import LatentDirichletAllocation

number_of_topics = 5

model = LatentDirichletAllocation(n_components=number_of_topics, random_state=0)


In [336]:
model.fit(tf)


LatentDirichletAllocation(n_components=5, random_state=0)

In [337]:
def display_topics(model, feature_names, no_top_words):
    topic_dict = {}
    for topic_idx, topic in enumerate(model.components_):
        topic_dict["Topic %d words" % (topic_idx)]= ['{}'.format(feature_names[i])
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
        topic_dict["Topic %d weights" % (topic_idx)]= ['{:.1f}'.format(topic[i])
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
    return pd.DataFrame(topic_dict)

no_top_words = 20
print("short_prompt_responses_Japanese_employee_1:")
display_topics(model, tf_feature_names, no_top_words)


short_prompt_responses_Japanese_employee_1:


,Topic 0 words,Topic 0 weights,Topic 1 words,Topic 1 weights,Topic 2 words,Topic 2 weights,Topic 3 words,Topic 3 weights,Topic 4 words,Topic 4 weights
0,commitment,0.2,dedication,2.2,respect,5.1,commitment,0.2,culture,6.1
1,desk,0.2,office,2.2,japan,4.2,desk,0.2,towards,5.2
2,dedication,0.2,respect,2.2,harmony,3.8,dedication,0.2,personal,5.2
3,team,0.2,become,2.2,company,3.2,team,0.2,company,4.2
4,precision,0.2,cacophony,2.2,sometimes,3.2,precision,0.2,harmony,3.6
5,way,0.2,principles,2.2,day,3.0,way,0.2,loyalty,3.6
6,traditional,0.2,commitment,1.2,life,2.8,traditional,0.2,workplace,3.6
7,office,0.2,desk,1.2,culture,2.3,office,0.2,time,3.4
8,respect,0.2,precision,1.2,team,2.2,respect,0.2,every,3.4
9,towards,0.2,traditional,1.2,hierarchy,2.2,towards,0.2,concept,3.2


In [338]:
from sklearn.feature_extraction.text import CountVectorizer

# Adjusted token pattern
token_pattern = r'\b[a-zA-Z]{3,}\b'  # Only words with 3 or more letters

vectorizer = CountVectorizer(max_df=0.9, min_df=0.1, token_pattern=token_pattern)


# apply transformation
tf1 = vectorizer.fit_transform(df1['clean_tweet']).toarray() #xth row te yth word thakle 1, 0-1 matrix

# tf_feature_names tells us what word each column in the matric represents
tf_feature_names = vectorizer.get_feature_names_out() #yth word as a list

In [339]:
len(tf1[0].tolist())

285

In [340]:
tf_feature_names[20]

'bonding'

In [341]:
from sklearn.decomposition import LatentDirichletAllocation

number_of_topics = 5

model1 = LatentDirichletAllocation(n_components=number_of_topics, random_state=0)

model1.fit(tf1)

no_top_words = 20
print("short_prompt_responses_Japanese_employee_2:")
display_topics(model1, tf_feature_names, no_top_words)

short_prompt_responses_Japanese_employee_2:


,Topic 0 words,Topic 0 weights,Topic 1 words,Topic 1 weights,Topic 2 words,Topic 2 weights,Topic 3 words,Topic 3 weights,Topic 4 words,Topic 4 weights
0,value,2.2,personal,3.2,typically,5.2,professional,2.2,many,0.2
1,cultural,2.2,harmony,2.2,highly,4.2,culture,2.2,teamwork,0.2
2,typically,2.2,usually,2.2,high,3.2,japan,2.2,workplace,0.2
3,teamwork,1.2,dedicated,2.2,individual,3.2,means,1.2,long,0.2
4,show,1.2,tend,2.2,company,3.2,teamwork,1.2,job,0.2
5,maintaining,1.2,terms,2.2,culture,3.2,language,1.2,high,0.2
6,place,1.2,efficiency,1.2,strong,2.2,due,1.2,common,0.2
7,take,1.2,development,1.2,known,2.2,many,1.2,polite,0.2
8,many,1.2,detail,1.2,common,2.2,coworkers,1.2,may,0.2
9,may,1.2,part,1.2,within,2.2,formal,1.2,harmony,0.2


In [342]:
from sklearn.feature_extraction.text import CountVectorizer

# Adjusted token pattern
token_pattern = r'\b[a-zA-Z]{3,}\b'  # Only words with 3 or more letters

vectorizer = CountVectorizer(max_df=0.9, min_df=0.1, token_pattern=token_pattern)


# apply transformation
tf2 = vectorizer.fit_transform(df2['clean_tweet']).toarray() #xth row te yth word thakle 1, 0-1 matrix

# tf_feature_names tells us what word each column in the matric represents
tf_feature_names = vectorizer.get_feature_names_out() #yth word as a list

In [343]:
len(tf2[0].tolist())

632

In [345]:
tf_feature_names[20]

'ambassador'

In [346]:
from sklearn.decomposition import LatentDirichletAllocation

number_of_topics = 5

model2 = LatentDirichletAllocation(n_components=number_of_topics, random_state=0)

model2.fit(tf2)

no_top_words = 20
print("short_prompt_responses_Japanese_international student_1:")
display_topics(model2, tf_feature_names, no_top_words)

short_prompt_responses_Japanese_international student_1:


,Topic 0 words,Topic 0 weights,Topic 1 words,Topic 1 weights,Topic 2 words,Topic 2 weights,Topic 3 words,Topic 3 weights,Topic 4 words,Topic 4 weights
0,like,4.2,home,4.2,way,3.2,japan,0.2,every,8.2
1,living,3.2,american,3.2,blend,2.2,tokyo,0.2,university,3.2
2,student,3.2,japan,2.2,see,2.2,culture,0.2,english,3.2
3,learning,2.2,culture,2.2,western,2.2,barriers,0.2,cultural,3.2
4,culture,2.2,back,2.2,differences,2.2,hello,0.2,student,3.2
5,sushi,2.2,sometimes,2.2,despite,2.2,learning,0.2,time,2.2
6,eyes,2.2,pushed,2.2,understand,2.2,student,0.2,grown,2.2
7,theres,2.2,conversations,2.2,english,2.2,experiences,0.2,academic,2.2
8,hand,2.2,exciting,2.2,traditions,2.2,western,0.2,heart,2.2
9,feel,2.2,learn,2.2,appreciate,2.2,journey,0.2,love,2.2


In [347]:
from sklearn.feature_extraction.text import CountVectorizer

# Adjusted token pattern
token_pattern = r'\b[a-zA-Z]{3,}\b'  # Only words with 3 or more letters

vectorizer = CountVectorizer(max_df=0.9, min_df=0.1, token_pattern=token_pattern)


# apply transformation
tf3 = vectorizer.fit_transform(df3['clean_tweet']).toarray() #xth row te yth word thakle 1, 0-1 matrix

# tf_feature_names tells us what word each column in the matric represents
tf_feature_names = vectorizer.get_feature_names_out() #yth word as a list

In [348]:
len(tf3[0].tolist())

303

In [349]:
tf_feature_names[20]

'arrangement'

In [350]:
from sklearn.decomposition import LatentDirichletAllocation

number_of_topics = 5

model3 = LatentDirichletAllocation(n_components=number_of_topics, random_state=0)

model3.fit(tf3)

no_top_words = 20
print("short_prompt_responses_Japanese_international student_2:")
display_topics(model3, tf_feature_names, no_top_words)

short_prompt_responses_Japanese_international student_2:


,Topic 0 words,Topic 0 weights,Topic 1 words,Topic 1 weights,Topic 2 words,Topic 2 weights,Topic 3 words,Topic 3 weights,Topic 4 words,Topic 4 weights
0,like,5.2,first,2.2,new,7.2,diligent,0.2,diligent,0.2
1,education,5.2,culture,1.9,students,5.2,exchange,0.2,exchange,0.2
2,students,4.2,language,1.7,language,3.2,education,0.2,education,0.2
3,countries,4.2,traditional,1.2,studies,3.2,likely,0.2,likely,0.2
4,struggle,4.2,adapt,1.2,culture,3.2,shock,0.2,shock,0.2
5,due,4.2,chosen,1.2,values,2.2,respect,0.2,respect,0.2
6,academic,3.2,dedication,1.2,structure,2.2,participate,0.2,participate,0.2
7,study,3.2,pursue,1.2,proficiency,2.2,unique,0.2,unique,0.2
8,homesickness,3.2,face,1.2,fellow,2.2,new,0.2,new,0.2
9,individual,3.2,environment,1.2,society,2.2,language,0.2,language,0.2


### United Arab Emriates

In [351]:
import pandas as pd
df = pd.read_csv('short_prompt_responses_Emiratis_empolyee_1.csv',error_bad_lines=False)
df1 = pd.read_csv('short_prompt_responses_Emiratis_empolyee_2.csv',error_bad_lines=False)
df2 = pd.read_csv('short_prompt_responses_Emiratis_international student_1.csv',error_bad_lines=False)
df3 = pd.read_csv('short_prompt_responses_Emiratis_international student_2.csv',error_bad_lines=False)

<ipython-input-351-8546fec29b48>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('short_prompt_responses_Emiratis_empolyee_1.csv',error_bad_lines=False)
<ipython-input-351-8546fec29b48>:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df1 = pd.read_csv('short_prompt_responses_Emiratis_empolyee_2.csv',error_bad_lines=False)
<ipython-input-351-8546fec29b48>:4: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df2 = pd.read_csv('short_prompt_responses_Emiratis_international student_1.csv',error_bad_lines=False)
<ipython-input-351-8546fec29b48>:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df3 = pd.r

In [352]:
import re
import string

def remove_links(tweet):
    '''Takes a string and removes web links from it'''
    tweet = re.sub(r'http\S+', '', tweet) # remove http links
    tweet = re.sub(r'bit.ly/\S+', '', tweet) # rempve bitly links
    tweet = tweet.strip('[link]') # remove [links]
    return tweet

def remove_users(tweet):
    '''Takes a string and removes retweet and @user information'''
    tweet = re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet) # remove retweet
    tweet = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet) # remove tweeted at
    return tweet

# Add custom words to stopwords list
additional_stopwords = {'also', '...', 'etc', 'often', ',','keep','might','morning','always','yet','find','make','sense','wake'}
my_stopwords = set(stopwords.words('english')) | additional_stopwords
# word_rooter = nltk.stem.snowball.PorterStemmer(ignore_stopwords=False).stem
my_punctuation = '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~•@'


# Define a translation table to remove punctuation
translator = str.maketrans('', '', string.punctuation)

def clean_tweet(tweet, bigrams=False):
    tweet = remove_users(tweet)
    tweet = remove_links(tweet)
    tweet = tweet.lower()  # Convert to lower case
    tweet = tweet.translate(translator)  # Remove punctuation
    tweet = re.sub('\s+', ' ', tweet)  # Remove double spacing
    tweet = re.sub('([0-9]+)', '', tweet)  # Remove numbers
    tweet_token_list = [word for word in tweet.split(' ')
                        if word not in my_stopwords]  # Remove stopwords
    # stemming algorithm sometimes cut off the end of the words which may lead to inaccuracy.
    # tweet_token_list = [word_rooter(word) if '#' not in word else word
    #                     for word in tweet_token_list] # apply word rooter

    if bigrams:
        tweet_token_list = tweet_token_list+[tweet_token_list[i]+'_'+tweet_token_list[i+1]
                                            for i in range(len(tweet_token_list)-1)]

    tweet = ' '.join(tweet_token_list)
    return tweet

# Apply the cleaning function to your dataframe
df['clean_tweet'] = df['response'].apply(clean_tweet)
df1['clean_tweet'] = df1['response'].apply(clean_tweet)
df2['clean_tweet'] = df2['response'].apply(clean_tweet)
df3['clean_tweet'] = df3['response'].apply(clean_tweet)

In [353]:
from sklearn.feature_extraction.text import CountVectorizer

# Adjusted token pattern
token_pattern = r'\b[a-zA-Z]{3,}\b'  # Only words with 3 or more letters

vectorizer = CountVectorizer(max_df=0.9, min_df=0.1, token_pattern=token_pattern)


# apply transformation
tf = vectorizer.fit_transform(df['clean_tweet']).toarray() #xth row te yth word thakle 1, 0-1 matrix

# tf_feature_names tells us what word each column in the matric represents
tf_feature_names = vectorizer.get_feature_names_out() #yth word as a list


In [354]:
len(tf[0].tolist())


481

In [355]:
tf_feature_names[20]

'appreciate'

In [356]:
from sklearn.decomposition import LatentDirichletAllocation

number_of_topics = 5

model = LatentDirichletAllocation(n_components=number_of_topics, random_state=0)


In [357]:
model.fit(tf)


LatentDirichletAllocation(n_components=5, random_state=0)

In [358]:
def display_topics(model, feature_names, no_top_words):
    topic_dict = {}
    for topic_idx, topic in enumerate(model.components_):
        topic_dict["Topic %d words" % (topic_idx)]= ['{}'.format(feature_names[i])
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
        topic_dict["Topic %d weights" % (topic_idx)]= ['{:.1f}'.format(topic[i])
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
    return pd.DataFrame(topic_dict)

no_top_words = 20
print("short_prompt_responses_Emiratis_employee_1:")
display_topics(model, tf_feature_names, no_top_words)


short_prompt_responses_Emiratis_employee_1:


,Topic 0 words,Topic 0 weights,Topic 1 words,Topic 1 weights,Topic 2 words,Topic 2 weights,Topic 3 words,Topic 3 weights,Topic 4 words,Topic 4 weights
0,importance,0.2,professional,5.2,emirati,5.2,emirati,6.2,importance,0.2
1,values,0.2,development,5.2,values,2.2,culture,5.2,values,0.2
2,time,0.2,life,3.2,time,2.2,day,5.2,time,0.2
3,colleagues,0.2,workplace,3.2,new,2.2,every,5.2,colleagues,0.2
4,attire,0.2,daily,3.2,respect,2.2,traditional,4.2,attire,0.2
5,kandura,0.2,proud,3.2,country,2.2,uae,4.2,kandura,0.2
6,personal,0.2,emirates,2.2,great,2.2,ambition,3.2,personal,0.2
7,worklife,0.2,united,2.2,invest,2.2,contribute,3.2,worklife,0.2
8,ambition,0.2,rich,2.2,importance,1.2,nation,3.2,ambition,0.2
9,family,0.2,innovation,2.2,kandura,1.2,part,3.2,family,0.2


In [359]:
from sklearn.feature_extraction.text import CountVectorizer

# Adjusted token pattern
token_pattern = r'\b[a-zA-Z]{3,}\b'  # Only words with 3 or more letters

vectorizer = CountVectorizer(max_df=0.9, min_df=0.1, token_pattern=token_pattern)


# apply transformation
tf1 = vectorizer.fit_transform(df1['clean_tweet']).toarray() #xth row te yth word thakle 1, 0-1 matrix

# tf_feature_names tells us what word each column in the matric represents
tf_feature_names = vectorizer.get_feature_names_out() #yth word as a list

In [360]:
len(tf1[0].tolist())

262

In [361]:
tf_feature_names[20]

'bring'

In [362]:
from sklearn.decomposition import LatentDirichletAllocation

number_of_topics = 5

model1 = LatentDirichletAllocation(n_components=number_of_topics, random_state=0)

model1.fit(tf1)

no_top_words = 20
print("short_prompt_responses_Emiratis_employee_2:")
display_topics(model1, tf_feature_names, no_top_words)

short_prompt_responses_Emiratis_employee_2:


,Topic 0 words,Topic 0 weights,Topic 1 words,Topic 1 weights,Topic 2 words,Topic 2 weights,Topic 3 words,Topic 3 weights,Topic 4 words,Topic 4 weights
0,skills,2.2,strong,2.2,emiratis,4.2,emirati,4.2,emirati,3.2
1,development,2.2,uaes,2.2,employees,2.2,many,3.2,could,3.2
2,work,2.2,culture,2.2,government,2.2,work,3.2,workforce,2.2
3,emiratis,2.2,national,2.2,jobs,2.2,employees,2.2,strong,2.2
4,international,1.2,value,2.2,values,1.2,skills,2.2,person,2.2
5,usually,1.2,work,2.2,initiatives,1.2,traditional,2.2,likely,2.2
6,cultural,1.2,emirati,2.2,roles,1.2,strong,2.2,work,2.2
7,values,1.2,technology,1.2,furthermore,1.2,sector,2.2,values,1.2
8,arabic,1.2,english,1.2,women,1.2,government,2.2,english,1.2
9,technology,1.2,arabic,1.2,islamic,1.2,commitments,1.2,technology,1.2


In [363]:
from sklearn.feature_extraction.text import CountVectorizer

# Adjusted token pattern
token_pattern = r'\b[a-zA-Z]{3,}\b'  # Only words with 3 or more letters

vectorizer = CountVectorizer(max_df=0.9, min_df=0.1, token_pattern=token_pattern)


# apply transformation
tf2 = vectorizer.fit_transform(df2['clean_tweet']).toarray() #xth row te yth word thakle 1, 0-1 matrix

# tf_feature_names tells us what word each column in the matric represents
tf_feature_names = vectorizer.get_feature_names_out() #yth word as a list

In [364]:
len(tf2[0].tolist())

501

In [365]:
tf_feature_names[20]

'among'

In [366]:
from sklearn.decomposition import LatentDirichletAllocation

number_of_topics = 5

model2 = LatentDirichletAllocation(n_components=number_of_topics, random_state=0)

model2.fit(tf2)

no_top_words = 20
print("short_prompt_responses_Emiratis_international student_1:")
display_topics(model2, tf_feature_names, no_top_words)

short_prompt_responses_Emiratis_international student_1:


,Topic 0 words,Topic 0 weights,Topic 1 words,Topic 1 weights,Topic 2 words,Topic 2 weights,Topic 3 words,Topic 3 weights,Topic 4 words,Topic 4 weights
0,language,0.2,values,4.2,cultures,4.2,journey,3.2,new,5.2
1,arabic,0.2,learn,3.2,traditional,3.2,abroad,2.2,emiratis,5.2
2,ambassador,0.2,like,3.2,learning,3.2,rich,2.2,experiences,4.2
3,environment,0.2,traditions,2.2,every,3.2,identity,2.2,home,4.2
4,values,0.2,wear,2.2,emiratis,3.2,proud,2.2,eager,2.2
5,cultures,0.2,nation,2.2,education,2.2,learning,2.2,attire,2.2
6,education,0.2,study,2.2,abroad,2.2,share,2.2,modernity,2.2
7,abroad,0.2,traditional,2.2,food,2.2,home,2.2,environment,2.2
8,traditional,0.2,perspectives,2.2,homeland,2.2,academic,2.2,knowledge,2.2
9,challenging,0.2,uae,2.2,values,2.2,intriguing,2.2,diverse,2.2


In [367]:
from sklearn.feature_extraction.text import CountVectorizer

# Adjusted token pattern
token_pattern = r'\b[a-zA-Z]{3,}\b'  # Only words with 3 or more letters

vectorizer = CountVectorizer(max_df=0.9, min_df=0.1, token_pattern=token_pattern)


# apply transformation
tf3 = vectorizer.fit_transform(df3['clean_tweet']).toarray() #xth row te yth word thakle 1, 0-1 matrix

# tf_feature_names tells us what word each column in the matric represents
tf_feature_names = vectorizer.get_feature_names_out() #yth word as a list

In [368]:
len(tf3[0].tolist())

317

In [369]:
tf_feature_names[20]

'ambitious'

In [370]:
from sklearn.decomposition import LatentDirichletAllocation

number_of_topics = 5

model3 = LatentDirichletAllocation(n_components=number_of_topics, random_state=0)

model3.fit(tf3)

no_top_words = 20
print("short_prompt_responses_Emiratis_international student_2:")
display_topics(model3, tf_feature_names, no_top_words)

short_prompt_responses_Emiratis_international student_2:


,Topic 0 words,Topic 0 weights,Topic 1 words,Topic 1 weights,Topic 2 words,Topic 2 weights,Topic 3 words,Topic 3 weights,Topic 4 words,Topic 4 weights
0,various,3.2,emirati,2.2,may,8.2,home,0.2,emirati,2.2
1,culture,3.2,education,2.2,emiratis,5.2,various,0.2,studying,2.2
2,could,2.2,uaes,2.2,students,5.2,could,0.2,sciences,2.2
3,study,2.2,may,2.2,foreign,3.2,traditional,0.2,clothing,2.2
4,islamic,2.2,values,1.2,cultural,3.2,hardworking,0.2,hardworking,1.2
5,experience,2.2,studies,1.2,education,3.2,values,0.2,traditional,1.2
6,academic,2.2,lifestyle,1.2,studying,3.2,emirati,0.2,educational,1.2
7,knowledge,2.2,code,1.2,university,2.2,global,0.2,adapt,1.2
8,new,2.2,dress,1.2,like,2.2,strong,0.2,seen,1.2
9,differences,1.2,traditional,1.2,culturally,2.2,arts,0.2,australia,1.2
